### Generating Synthetic Translation data of German-French sentences for LLM-fine tuning

We use the model Llama 3.3 70B Instruct Turbo provided free of cost (at the time of this work) by Together AI.
https://www.together.ai/models/llama-3-3-70b-free

This model is chosen as it a multilingual large generative model that supports both German and French.

However, there are limits for the number of requests that can be sent per minute. 


In [11]:
import os
from together import Together
from dotenv import load_dotenv

In [12]:
load_dotenv()
together_ai_key = os.getenv('together_ai')

In [14]:

client = Together(api_key=together_ai_key)

model="meta-llama/Llama-3.3-70B-Instruct-Turbo-Free"
prompt = "Who are you"

response = client.chat.completions.create(
    model=model,
    messages=[{"role": "user", "content": prompt}],
)
print(response.choices[0].message.content)

I'm an artificial intelligence model known as Llama. Llama stands for "Large Language Model Meta AI."


In [20]:
prompt = f"""Generate a sentence in German and then give its French Translation.
Follow this format: German_sentence: , French_sentence:"""

response = client.chat.completions.create(
    model=model,
    messages=[{"role": "user", "content": prompt}],
)
print(response.choices[0].message.content)

Die Stadt Paris ist bekannt für ihre wunderschönen Sehenswürdigkeiten: , La ville de Paris est connue pour ses merveilleux sites touristiques.


In [32]:
prompt = f"""Generate 15 sentences in German and then give its French Translation. The sentences can be long or short.
Follow this format and keep the prefixes German: and French:. Directly give the examples in this format.
num_1: German_sentence: , French_sentence:  
num_2: German_sentence: , French_sentence: 
and so on until num_20"""

response = client.chat.completions.create(
    model=model,
    messages=[{"role": "user", "content": prompt}],
)
print(response.choices[0].message.content)

1: German: Die Katze ist sehr freundlich, French: Le chat est très gentil 
2: German: Ich gehe jeden Tag ins Fitnessstudio, French: Je vais à la salle de sport tous les jours 
3: German: Die Blumen sind sehr schön, French: Les fleurs sont très belles 
4: German: Ich liebe es, am Meer zu entspannen, French: J'adore me détendre au bord de la mer 
5: German: Mein Bruder ist sehr intelligent, French: Mon frère est très intelligent 
6: German: Die Stadt ist sehr groß und laut, French: La ville est très grande et bruyante 
7: German: Ich esse gerne Pizza, French: J'adore manger de la pizza 
8: German: Die Berge sind sehr hoch, French: Les montagnes sont très hautes 
9: German: Ich bin sehr müde heute, French: Je suis très fatigué aujourd'hui 
10: German: Die Musik ist sehr schön, French: La musique est très belle 
11: German: Ich gehe gerne ins Kino, French: J'adore aller au cinéma 
12: German: Die Sonne scheint sehr hell, French: Le soleil brille très fort 
13: German: Ich liebe es, mit Fre

We need atleast 2000 examples. Also, the free version limits 6 requests per minute (RPM). we need to make sure that we do not exceed the RPM.
If we generate 20 examples per request, we need to send approx. 2000/(20) = 100 requests in total.
This would take atleast (100/6): 16-17 min approx.


Sometimes, the LLM repeats the same sentence. Hence, to be on the safer side, we will generate 5000 sentences. 
i.e 250 requests.

But, first we will generate the translations in batches. We can provide a theme and generate sentences around it, so that we have variety in our sentences. 

In [ ]:
prompt_15 = f"""Generate 20 sentences in German and then give its French Translation. The sentences can be long or short.
Follow this format and keep the prefixes German: and French:. Directly give the examples in this format.
num_1: German_sentence: , French_sentence:  
num_2: German_sentence: , French_sentence: 
and so on until num_20"""

In [40]:
from tqdm import tqdm
import time

syn_data_dict = {}

start_time = time.time()   #start time
delay = 10                  # time delay between each requests


for iteration in tqdm(range(100)):
    # Ensure the delay of 'delay' seconds between requests
    if iteration > 0:
        time.sleep(delay)
    
    response = client.chat.completions.create(
    model=model,
    messages=[{"role": "user", "content": prompt_15}],
    top_p=0.7,
    temperature= 0.7,
    repetition_penalty= 1.2,
    
)
    syn_data_dict[iteration] = response.choices[0].message.content
    
end_time = time.time()
print(f"Total time taken: {end_time - start_time:.2f} seconds")


100%|██████████| 100/100 [25:39<00:00, 15.40s/it]

Total time taken: 1539.76 seconds


In [41]:
syn_data_dict_general = syn_data_dict
syn_data_dict_general

{0: "1: German: Die Katze ist sehr freundlich, French: Le chat est très gentil,\n2: German: Ich esse jeden Tag ein Stück Brot zum Frühstück, French: Je mange chaque jour un morceau de pain pour le petit déjeuner,\n3: German: Der Hund läuft schnell im Park, French: Le chien court vite dans le parc,\n4: German: Wir fahren morgen nach Paris, French: Nous partons demain pour Paris,\n5: German: Das Wetter ist heute schön, French: Le temps est beau aujourd'hui,\n6: German: Ich habe einen Bruder und eine Schwester, French: J'ai un frère et une sœur,\n7: German: Die Blumen sind rot und gelb, French: Les fleurs sont rouges et jaunes,\n8: German: Mein Lieblingsessen ist Pizza, French: Mon plat préféré est la pizza,\n9: German: Der Film beginnt um 20 Uhr, French: Le film commence à 20 heures,\n10: German: Ich spreche Deutsch und Englisch, French: Je parle allemand et anglais,\n11: German: Die Stadt ist groß und laut, French: La ville est grande et bruyante,\n12: German: Wir haben viel Spaß am Str

Now, lets create some sentences with specific thema. 

In [44]:
prompt_20_1 = f"""Generate 20 sentences in German and then give its French Translation. The sentences can be long or short.
The theme should be University, relationship, friends and office.
Follow this format and keep the prefixes German: and French:. Directly give the examples in this format.
num_1: German_sentence: , French_sentence:  
num_2: German_sentence: , French_sentence: 
and so on until num_20"""

response = client.chat.completions.create(
    model=model,
    messages=[{"role": "user", "content": prompt_20_1}],
)
print(response.choices[0].message.content)

1: German: Ich studiere an der Universität und habe viele Freunde dort, French: J'étudie à l'université et j'ai beaucoup d'amis là-bas,
2: German: Meine Freundin arbeitet im Büro und ich besuche sie oft, French: Ma petite amie travaille au bureau et je la visite souvent,
3: German: Die Universität ist sehr groß und hat viele verschiedene Fakultäten, French: L'université est très grande et a de nombreuses facultés différentes,
4: German: Ich habe mich mit meinen Freunden im Park verabredet, um über unsere Beziehungen zu sprechen, French: Je me suis donné rendez-vous avec mes amis au parc pour discuter de nos relations,
5: German: Mein Chef ist sehr streng und erwartet viel von seinen Mitarbeitern, French: Mon patron est très sévère et attend beaucoup de ses employés,
6: German: Ich liebe es, mit meinen Freunden zu feiern und neue Erinnerungen zu schaffen, French: J'adore faire la fête avec mes amis et créer de nouveaux souvenirs,
7: German: Die Beziehungen zwischen den Kollegen im Büro 

The prompt style works !!! Now, lets generate batch of these.

In [ ]:
def get_translation_dict(client,model,prompt, total_requests):
    """
    Takes in client object, model name and a prompt.
    Queries the model with the prompt for 'total_requests'
    Has a delay of 10s inbetween the requests. 

    Args:
        client (_type_): _description_
        model (str): name of the model
        prompt (str): prompt or query for the LLM
        total_requests (int) : number of requests to send 

    Returns a dict of {request_number: response}
    """
    syn_data_dict = {}

    start_time = time.time()   #start time
    delay = 10                  # time delay between each requests


    for iteration in tqdm(range(total_requests)):
        # Ensure the delay of 'delay' seconds between requests
        if iteration > 0:
            time.sleep(delay)
        
        response = client.chat.completions.create(
        model=model,
        messages=[{"role": "user", "content": prompt}],
        top_p=0.7,
        temperature= 0.7,
        repetition_penalty= 1.2,
        
    )
        syn_data_dict[iteration] = response.choices[0].message.content
        
    end_time = time.time()
    print(f"Total time taken: {end_time - start_time:.2f} seconds")

    return syn_data_dict
    

In [46]:
prompt_20_1 = f"""Generate 20 sentences in German and then give its French Translation. The sentences can be long or short.
The theme should be University, relationship, friends and office.
Follow this format and keep the prefixes German: and French:. Directly give the examples in this format.
num_1: German_sentence: , French_sentence:  
num_2: German_sentence: , French_sentence: 
and so on until num_20"""

syn_data_dict_thema1 = get_translation_dict(client=client, model=model,prompt= prompt_20_1, total_requests=100)

100%|██████████| 100/100 [33:37<00:00, 20.18s/it]

Total time taken: 2017.87 seconds


In [52]:
print(syn_data_dict_thema1[0])  # first value of dict


1: German: Ich studiere an der Universität und habe viele Freunde dort., French: J'étudie à l'université et j'ai beaucoup d'amis là-bas.
2: German: Meine beste Freundin arbeitet im Büro nebenan., French: Ma meilleure amie travaille dans le bureau à côté.
3: German: Wir verbringen unsere freie Zeit oft zusammen mit unseren Kollegen aus dem Büro., French: Nous passons souvent notre temps libre ensemble avec nos collègues du bureau.
4: German: Die Universitätsbibliothek ist mein Lieblingsort zum Lernen., French: La bibliothèque universitaire est mon endroit préféré pour étudier.
5: German: Mein Professor ist sehr nett und hilft mir bei allen Fragen., French: Mon professeur est très gentil et m'aide à toutes les questions.
6: German: Ich bin froh, dass ich solche netten Kollegen im Büro habe., French: Je suis heureux d'avoir de tels collègues agréables au bureau.
7: German: Unsere Gruppe von Freunden trifft sich jeden Wochenende., French: Notre groupe d'amis se réunit chaque week-end.
8: G

In [ ]:
def generate_df_from_dict(translation_dict, end_number):
    """
    Takes a dict of the form {iteration: translation pair of sentences upto n numbers(end_number)}
    
    returns a dataframe of sentences with 2 columns 'German' and 'French' (corresponding translations in French)

    Args:
        translation_dict (_type_): dict
        end_number (_type_): number of translation pairs in each value of the dict.

    Returns:
        _type_: pandas dataframe
    """
    german_sentences = []
    french_sentences = []

    # Iterate through each value in the dictionary
    for value in translation_dict.values():
        # Split by newlines to get individual translation pairs
        translation_pairs = value.split('\n')
        
        for pair in translation_pairs:
            if not pair.strip() or '{end_number}' in pair:
                continue
            
            # Extract the German and French parts
            try:
                # Split at ", French: " to isolate the German and French sentences
                german_part, french_part = pair.split(", French: ")
                
                # Remove the prefix before "German: " and strip any excess whitespace
                german_sentence = german_part.split("German: ")[1].strip()
                french_sentence = french_part.strip()
                
                # Append to the lists
                german_sentences.append(german_sentence)
                french_sentences.append(french_sentence)
            
            except ValueError:
                print(f"Skipping malformed entry: {pair}")

    # Create a DataFrame from the lists
    df = pd.DataFrame({
        "German": german_sentences,
        "French": french_sentences
    })
    
    return df
        

In [54]:
syn_data_df_thema1 = generate_df_from_dict(translation_dict= syn_data_dict_thema1, end_number=20)

Skipping malformed entry: 13: German: Ich habe mein Studium an der Universität erfolgreich abgeschlossen., French : J'ai terminé avec succès mes études à l'université.
Skipping malformed entry: 16: German: Ich bin dankbar für meine Freundschaften und die Erfahrungen, die ich gemacht habe., French : Je suis reconnaissant pour mes amitiés et les expériences que j'ai vécues.


In [57]:
def print_df_stats(df):
    print('Number of entries (rows, columns):', df.shape)
    print('Unique German sentences =',len(set(df['German'])))
    print('Unique French sentences =',len(set(df['French'])))



In [58]:
print_df_stats(syn_data_df_thema1)

Number of entries (rows, columns): (1998, 2)
Unique German sentences = 1281
Unique French sentences = 1337


In [59]:
syn_data_df_general = generate_df_from_dict(translation_dict= syn_data_dict_general, end_number=20)

Skipping malformed entry: 1: Ich gehe jeden Tag zum Markt, um frische Lebensmittel zu kaufen., Je vais tous les jours au marché pour acheter des produits frais.
Skipping malformed entry: 2: Die neue Regelung wird voraussichtlich im nächsten Monat in Kraft treten., La nouvelle réglementation devrait entrer en vigueur le mois prochain.
Skipping malformed entry: 3: Mein Bruder studiert Medizin an der Universität und möchte einmal Arzt werden., Mon frère étudie la médecine à l'université et veut devenir médecin un jour.
Skipping malformed entry: 4: Wir haben gestern Abend ein wunderschönes Konzert besucht., Nous avons assisté hier soir à un magnifique concert.
Skipping malformed entry: 5: Das Wetter war heute Morgen sehr schön, aber jetzt regnet es., Le temps était très beau ce matin, mais maintenant il pleut.
Skipping malformed entry: 6: Meine Schwester liebt es, Bücher über Geschichte zu lesen., Ma sœur adore lire des livres sur l'histoire.
Skipping malformed entry: 7: Der Zug ist gerade

In [60]:
print_df_stats(syn_data_df_general)

Number of entries (rows, columns): (1470, 2)
Unique German sentences = 473
Unique French sentences = 521


In [63]:
syn_data_df_thema1.to_csv('syn_data/syn_thema1.csv',index=False, encoding='utf-8')
syn_data_df_general.to_csv('syn_data/syn_general.csv',index=False, encoding='utf-8')
combined_df = pd.concat([syn_data_df_thema1, syn_data_df_general], ignore_index=True)
combined_df.to_csv('syn_data/combined_data.csv',index=False, encoding='utf-8')

##Make sure to set encoding as 'utf-8' so that the German umlauts and french special characters are retained.
## make sure to use the same encoding when loading them too.

In [64]:
print_df_stats(combined_df)

Number of entries (rows, columns): (3468, 2)
Unique German sentences = 1754
Unique French sentences = 1858


In [65]:
prompt_20_2 = f"""Generate 20 sentences in German and then give its French Translation. The sentences can be long or short.
The theme should be Traveling, Sports, Education and entertainment.
Follow this format and keep the prefixes German: and French:. Directly give the examples in this format.
num_1: German_sentence: , French_sentence:  
num_2: German_sentence: , French_sentence: 
and so on until num_20"""

syn_data_dict_thema2 = get_translation_dict(client=client, model=model,prompt= prompt_20_2, total_requests=100)

100%|██████████| 100/100 [34:21<00:00, 20.62s/it]

Total time taken: 2061.89 seconds


In [66]:
syn_data_df_thema2 = generate_df_from_dict(translation_dict= syn_data_dict_thema2, end_number=20)

Skipping malformed entry: 16: German: Die Deutsche Bahn ist ein wichtiges Verkehrsmittel in Deutschland., French : La Deutsche Bahn est un moyen de transport important en Allemagne.
Skipping malformed entry: 1. **German**: Ich liebe es zu reisen und neue Länder zu erkunden., **French**: J'adore voyager et découvrir de nouveaux pays.
Skipping malformed entry: 2. **German**: Der Fußball ist ein sehr beliebter Sport in Deutschland., **French**: Le football est un sport très populaire en Allemagne.
Skipping malformed entry: 3. **German**: Die Bildung ist der Schlüssel zum Erfolg in unserem Leben., **French**: L'éducation est la clé du succès dans notre vie.
Skipping malformed entry: 4. **German**: Ich bin ein großer Fan von Actionfilmen und sehne mich nach dem nächsten Blockbuster., **French**: Je suis un grand fan de films d'action et j'attends avec impatience le prochain blockbuster.
Skipping malformed entry: 5. **German**: Reisen kann sehr teuer sein, aber es lohnt sich immer., **French

In [67]:
print_df_stats(syn_data_df_thema2)

Number of entries (rows, columns): (1978, 2)
Unique German sentences = 1262
Unique French sentences = 1326


In [68]:
syn_data_df_thema2.to_csv('syn_data/syn_thema2.csv',index=False, encoding='utf-8')

combined_df_2 = pd.concat([combined_df, syn_data_df_thema2], ignore_index=True)
combined_df_2.to_csv('syn_data/combined_data_2.csv',index=False, encoding='utf-8')

In [69]:
print_df_stats(combined_df_2)

Number of entries (rows, columns): (5446, 2)
Unique German sentences = 3015
Unique French sentences = 3183


#### Now, we have around 3000 Unique translations. So, we can stop here. 
We will drop the duplicates from 'German' column and save the df. Later, when loading the df, we can select random 2000 examples. 

In [74]:
combined_df_cleaned = combined_df_2.drop_duplicates(subset='German')
combined_df_cleaned.head()


,German,French
0,Ich studiere an der Universität und habe viele...,J'étudie à l'université et j'ai beaucoup d'ami...
1,Meine beste Freundin arbeitet im Büro nebenan.,Ma meilleure amie travaille dans le bureau à c...
2,Wir verbringen unsere freie Zeit oft zusammen ...,Nous passons souvent notre temps libre ensembl...
3,Die Universitätsbibliothek ist mein Lieblingso...,La bibliothèque universitaire est mon endroit ...
4,Mein Professor ist sehr nett und hilft mir bei...,Mon professeur est très gentil et m'aide à tou...


In [76]:
combined_df_cleaned.to_csv('syn_data/syn_data_3000.csv',index=False, encoding='utf-8')

The random selection of 2000 pairs is done in next step.